In [2]:
from analysis2014.process_2014_data import get_image_vec_list
item_dict = get_image_vec_list()
item_mat = list(item_dict.values())
num_item_feat = len(item_mat[0])

processed 1000 asins
processed 2000 asins
processed 3000 asins
processed 4000 asins
processed 5000 asins
processed 6000 asins


In [3]:
import pandas as pd
import json
with open("analysis2014/baby_5_core_reviews.json", "r") as pvr_file:
    pv_rev = json.load(pvr_file)
df = pd.DataFrame(pv_rev)

In [4]:
counter = 0
r_id_r_int_mapping = {}
for r_id in pd.unique(df["reviewer_id"]):
    if r_id not in r_id_r_int_mapping:
        r_id_r_int_mapping[r_id] = counter
        counter += 1

In [5]:
counter = 0
p_id_p_int_mapping = {}
for p_id in pd.unique(df["asin"]):
    if p_id not in p_id_p_int_mapping:
        p_id_p_int_mapping[p_id] = counter
        counter += 1

In [6]:
df["reviewer_id"] = df["reviewer_id"].map(r_id_r_int_mapping)
df["item_id"] = df["asin"].map(p_id_p_int_mapping)


In [7]:
from cdl_model import CDL
model = CDL(item_mat, [num_item_feat, 512, 64], item_dict)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [8]:
# This should only be done once
model.pre_train_item_representation(lambda_w=0.001, encoder_noise=0.3, epochs=10)
trained_encoders = model.trained_encoders
trained_decoders = model.trained_decoders

Epoch 1/10
110/110 - 18s - loss: 1.3424
Epoch 2/10
110/110 - 5s - loss: 1.2267
Epoch 3/10
110/110 - 5s - loss: 1.2267
Epoch 4/10
110/110 - 5s - loss: 1.2268
Epoch 5/10
110/110 - 5s - loss: 1.2268
Epoch 6/10
110/110 - 5s - loss: 1.2268
Epoch 7/10
110/110 - 5s - loss: 1.2268
Epoch 8/10
110/110 - 5s - loss: 1.2267
Epoch 9/10
110/110 - 5s - loss: 1.2267
Epoch 10/10
110/110 - 6s - loss: 1.2267
Epoch 1/10
110/110 - 1s - loss: 0.1001
Epoch 2/10
110/110 - 0s - loss: 0.0295
Epoch 3/10
110/110 - 0s - loss: 0.0294
Epoch 4/10
110/110 - 0s - loss: 0.0294
Epoch 5/10
110/110 - 0s - loss: 0.0294
Epoch 6/10
110/110 - 0s - loss: 0.0295
Epoch 7/10
110/110 - 0s - loss: 0.0295
Epoch 8/10
110/110 - 0s - loss: 0.0294
Epoch 9/10
110/110 - 0s - loss: 0.0294
Epoch 10/10
110/110 - 0s - loss: 0.0295


In [9]:
model.trained_encoders = trained_encoders
model.trained_decoders = trained_decoders

In [10]:
train_mat = df.sample(frac=0.8,random_state=200) #random state is a seed value
test_mat = df.drop(train_mat.index)
model_history = model.train_cdl_model(train_mat, test_mat, lambda_u=0.01, lambda_v=0.1, lambda_n=0.1, epochs=3)
testing_rmse = model.get_rmse(test_mat)
print('Testing RMSE = {}'.format(testing_rmse))

Epoch 1/3
2010/2010 [==============================] - 179s 85ms/step - loss: 29946.6331 - dot_layer_loss: 28.7331 - model_2_loss: 1.2705 - val_loss: 656.6857 - val_dot_layer_loss: 0.7739 - val_model_2_loss: 1.2730

In [11]:
test_user, test_item, test_item_feat, test_label = model.get_input_and_labels(test_mat.iloc[:,])

In [12]:
pred_out = model.cdl_model.predict([test_user, test_item, test_item_feat])
print(test_label.flatten())
print(pred_out[0].flatten())

[1.  0.8 1.  ... 0.8 1.  1. ]
[-0.02985503  0.01920876 -0.03073503 ... -0.03096009  0.09094845
  0.03096759]


In [13]:
test_labels_flat = test_label.flatten()
rating_predictions = pred_out[0].flatten()
tuples = [(test_labels_flat[i], rating_predictions[i]) for i in range(len(test_labels_flat))]

for i in [30, 50, 100, 300]:
    recommendations = sorted(tuples, key=lambda x: x[1], reverse=True)[:i]
    prediction_average = sum([x[0] for x in recommendations])/len(recommendations)
    dataset_mean = test_labels_flat.mean()
    print("The average rating of the top {} recommended products is: {}".format(i, round(prediction_average, 2)))
    print("The average rating of the entire test dataset is: ", round(dataset_mean, 2))
    print("The improvement percentage is: {}% \n".format(round((prediction_average-dataset_mean)*100), 5))


The average rating of the top 30 recommended products is: 0.88
The average rating of the entire test dataset is:  0.84
The improvement percentage is: 4% 

The average rating of the top 50 recommended products is: 0.9
The average rating of the entire test dataset is:  0.84
The improvement percentage is: 6% 

The average rating of the top 100 recommended products is: 0.9
The average rating of the entire test dataset is:  0.84
The improvement percentage is: 6% 

The average rating of the top 300 recommended products is: 0.86
The average rating of the entire test dataset is:  0.84
The improvement percentage is: 2% 

